In [1]:
from langchain_google_cloud_sql_pg import (
    PostgresEngine,
    PostgresSaver,
)
from src.config import env
from langchain_core.runnables import RunnableConfig
from langchain.load import dumpd
import os

In [ ]:
async def get_ckeckpoint():
    engine = await PostgresEngine.afrom_instance(
        project_id=env.PROJECT_ID,
        region=env.LOCATION,
        instance=env.INSTANCE,
        database=env.DATABASE,
        user=env.DATABASE_USER,
        password=env.DATABASE_PASSWORD,
        engine_args={"pool_pre_ping": True, "pool_recycle": 300},
    )
    checkpointer = await PostgresSaver.create(engine=engine)
    return checkpointer


checkpointer = await get_ckeckpoint()

In [3]:
user_id = "5521996143632"
config = RunnableConfig(configurable={"thread_id": user_id})
l = await checkpointer.aget(config=config)

In [9]:
l["channel_values"]["messages"][5:]

[AIMessage(content='Olá! Sinto muito que você esteja se sentindo mal. Para te ajudar da melhor forma, preciso de algumas informações.\n\nVocê está buscando um atendimento de emergência ou uma consulta de rotina?\nVocê pode me dizer qual o seu endereço para que eu possa verificar a unidade de saúde mais próxima?', additional_kwargs={'timestamp': '2025-08-15T21:36:39.725915+00:00'}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 5999, 'candidates_token_count': 61, 'total_token_count': 6143, 'prompt_tokens_details': [{'modality': 1, 'token_count': 5999}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 61}], 'thoughts_token_count': 83, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.8219513189597208, 'model_name': 'gemini-2.5-flash'}, id='run--85218041-5bd8-43cb-a1b5-68ad1e0642e9-0', usage_metadata={'input_tokens': 5999, 'output_tokens': 61, 'total_tokens': 6143, 

In [ ]:
from enum import Enum
from typing import Sequence
from langchain_core.messages import AIMessage, BaseMessage, ToolMessage


class ErrorCode(Enum):
    GRAPH_RECURSION_LIMIT = "GRAPH_RECURSION_LIMIT"
    INVALID_CONCURRENT_GRAPH_UPDATE = "INVALID_CONCURRENT_GRAPH_UPDATE"
    INVALID_GRAPH_NODE_RETURN_VALUE = "INVALID_GRAPH_NODE_RETURN_VALUE"
    MULTIPLE_SUBGRAPHS = "MULTIPLE_SUBGRAPHS"
    INVALID_CHAT_HISTORY = "INVALID_CHAT_HISTORY"


def create_error_message(*, message: str, error_code: ErrorCode) -> str:
    return (
        f"{message}\n"
        "For troubleshooting, visit: https://python.langchain.com/docs/"
        f"troubleshooting/errors/{error_code.value}"
    )


def _validate_chat_history(
    messages: Sequence[BaseMessage],
):
    """Validate that all tool calls in AIMessages have a corresponding ToolMessage."""
    all_tool_calls = [
        tool_call
        for message in messages
        if isinstance(message, AIMessage)
        for tool_call in message.tool_calls
    ]
    tool_call_ids_with_results = {
        message.tool_call_id for message in messages if isinstance(message, ToolMessage)
    }
    tool_calls_without_results = [
        tool_call
        for tool_call in all_tool_calls
        if tool_call["id"] not in tool_call_ids_with_results
    ]

    return all_tool_calls